# Fire

## 4. Feature Engineering
### Collect & clean weather data

In [1]:
import pandas as pd
import requests
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
import src.scripts.Preprocessing

ModuleNotFoundError: No module named 'src'

In [2]:
data = pd.read_csv('../../data/H_Fire_PNT.csv')

In [3]:
data.head(2)

,FIRE_NO,FIRE_YEAR,IGN_DATE,FIRE_CAUSE,FIRELABEL,FRCNTR,ZONE,FIRE_ID,FIRE_TYPE,GEO_DESC,LATITUDE,LONGITUDE,SIZE_HA,FCODE,SHAPE,OBJECTID,X,Y
0,C30140,2014,2.014080e+13,Lightning,2014-C30140,7.0,3.0,700140,Fire,North Arm Quesnel Lake,52.7163,-120.8649,0.009,JA70003000,NaN,16015562,-120.864937,52.716331
1,C40162,2003,2.003071e+13,Lightning,2003-C40162,7.0,4.0,700162,Fire,Young Lake,51.3333,-120.8167,0.009,JA70003000,NaN,16015563,-120.816667,51.333333


In [6]:
#select the relevant cols 
weather_df = data[['fl_date', 'origin_city_name','dest_city_name']]
weather_df.head()

,fl_date,origin_city_name,dest_city_name
0,2018-07-10,"San Antonio, TX","Nashville, TN"
1,2019-10-10,"Chicago, IL","Orlando, FL"
2,2019-07-18,"Lawton/Fort Sill, OK","Dallas/Fort Worth, TX"
3,2018-11-25,"Savannah, GA","Chicago, IL"
4,2018-10-24,"Detroit, MI","Santa Ana, CA"


## Create table of unique date/city combinations
To reduce the number of API calls

In [7]:
city_date = flights[['fl_date', 'origin_city_name']].rename({'origin_city_name': 'city'}, axis = 1)    #start with origin cities
temp = flights[['fl_date', 'dest_city_name']].rename({'dest_city_name': 'city'}, axis = 1)             #dest cities

In [8]:
#append dest cities to origin cities
city_date = pd.concat([city_date, temp], ignore_index=True)

In [9]:
city_date.shape        #confirm the df doubles in size

(20000, 2)

In [10]:
#find unique city_date combos
city_date_unique = city_date.groupby(['fl_date','city']).size().reset_index().rename(columns={0:'count'}).sort_values(by = 'count', ascending = False)

In [11]:
#add empty weather col
city_date_unique['weather_categ'] = 'empty'

In [12]:
city_date_unique.reset_index(drop = True, inplace= True)

In [13]:
city_date_unique

,fl_date,city,count,weather_categ
0,2018-10-25,"New York, NY",7,empty
1,2019-07-10,"Chicago, IL",7,empty
2,2019-10-02,"Chicago, IL",7,empty
3,2019-07-01,"Chicago, IL",7,empty
4,2019-11-23,"Denver, CO",7,empty
...,...,...,...,...
15625,2018-10-05,"Baltimore, MD",1,empty
15626,2018-10-05,"Boston, MA",1,empty
15627,2018-10-05,"Charlotte, NC",1,empty
15628,2018-10-05,"Fort Lauderdale, FL",1,empty


In [14]:
city_date_unique.shape

(15630, 4)

We have reduce 20 000 combinations to 15 630.

## Import lat long table
uscities.csv is from https://simplemaps.com/data/us-cities

In [15]:
lat_long_df = pd.read_csv('data/uscities.csv')
lat_long_df = lat_long_df[['city', 'state_id', 'lat', 'lng', 'timezone']]

In [16]:
lat_long_df.shape

(60818, 5)

In [17]:
lat_long_df

,city,state_id,lat,lng,timezone
0,New York,NY,40.6943,-73.9249,America/New_York
1,Los Angeles,CA,34.1141,-118.4068,America/Los_Angeles
2,Chicago,IL,41.8375,-87.6866,America/Chicago
3,Miami,FL,25.7840,-80.2101,America/New_York
4,Dallas,TX,32.7935,-96.7667,America/Chicago
...,...,...,...,...,...
60813,30404,ID,"Drummond, ID",NaN,NaN
60814,30405,WY,"Lost Springs, WY",NaN,NaN
60815,30406,SD,"Provo, SD",NaN,NaN
60816,30407,MT,"Goldcreek, MT",NaN,NaN


In [18]:
def city_state_creator(row):
    return str(row['city']) + ", " + row['state_id']

In [19]:
#adding city state to the whole data frame
lat_long_df['city_state'] = lat_long_df.apply(city_state_creator, axis = 1)           #pass in func object, not calling the func

In [20]:
lat_long_df

,city,state_id,lat,lng,timezone,city_state
0,New York,NY,40.6943,-73.9249,America/New_York,"New York, NY"
1,Los Angeles,CA,34.1141,-118.4068,America/Los_Angeles,"Los Angeles, CA"
2,Chicago,IL,41.8375,-87.6866,America/Chicago,"Chicago, IL"
3,Miami,FL,25.7840,-80.2101,America/New_York,"Miami, FL"
4,Dallas,TX,32.7935,-96.7667,America/Chicago,"Dallas, TX"
...,...,...,...,...,...,...
60813,30404,ID,"Drummond, ID",NaN,NaN,"30404, ID"
60814,30405,WY,"Lost Springs, WY",NaN,NaN,"30405, WY"
60815,30406,SD,"Provo, SD",NaN,NaN,"30406, SD"
60816,30407,MT,"Goldcreek, MT",NaN,NaN,"30407, MT"


dump the last rows of the df that do not match the format:

In [21]:
lat_long_df.drop(lat_long_df.tail(30409).index,inplace=True)

In [22]:
lat_long_df.tail()

,city,state_id,lat,lng,timezone,city_state
30404,Drummond,ID,43.9996,-111.3433,America/Boise,"Drummond, ID"
30405,Lost Springs,WY,42.7652,-104.9255,America/Denver,"Lost Springs, WY"
30406,Provo,SD,43.1937,-103.8329,America/Denver,"Provo, SD"
30407,Goldcreek,MT,46.5838,-112.9284,America/Denver,"Goldcreek, MT"
30408,Le Roy,IA,40.8779,-93.5926,America/Chicago,"Le Roy, IA"


In [23]:
#let's save this clean table as csv
lat_long_df.to_csv('data/uscities_ll.csv', header=True, index=False)

## Now can we prepare to call the API

In [24]:
#api requires lat/long, found a csv of american cities and lat/longs
latlong_df = pd.read_csv('data/uscities_ll.csv')

In [25]:
latlong_df.shape

(30409, 6)

In [26]:
latlong_df

,city,state_id,lat,lng,timezone,city_state
0,New York,NY,40.6943,-73.9249,America/New_York,"New York, NY"
1,Los Angeles,CA,34.1141,-118.4068,America/Los_Angeles,"Los Angeles, CA"
2,Chicago,IL,41.8375,-87.6866,America/Chicago,"Chicago, IL"
3,Miami,FL,25.7840,-80.2101,America/New_York,"Miami, FL"
4,Dallas,TX,32.7935,-96.7667,America/Chicago,"Dallas, TX"
...,...,...,...,...,...,...
30404,Drummond,ID,43.9996,-111.3433,America/Boise,"Drummond, ID"
30405,Lost Springs,WY,42.7652,-104.9255,America/Denver,"Lost Springs, WY"
30406,Provo,SD,43.1937,-103.8329,America/Denver,"Provo, SD"
30407,Goldcreek,MT,46.5838,-112.9284,America/Denver,"Goldcreek, MT"


In [27]:
latlong_df['city_state'].nunique()

30351

In [28]:
#drop the duplicate city_states
latlong_df = latlong_df.drop_duplicates(subset='city_state', keep="first")

In [29]:
latlong_df.shape

(30351, 6)

### API functions

In [30]:
from src.modules.data_preprocessing import open_weather_api, city_scrubber, weather_condition

In [31]:
#test api function
test_api = open_weather_api("41.75","-87.75","2018-10-25", "America%2FNew_York")
test_api

{'latitude': 41.75,
 'longitude': -87.75,
 'generationtime_ms': 0.34308433532714844,
 'utc_offset_seconds': -14400,
 'timezone': 'America/New_York',
 'timezone_abbreviation': 'EDT',
 'elevation': 192.0,
 'hourly_units': {'time': 'iso8601',
  'precipitation': 'mm',
  'rain': 'mm',
  'snowfall': 'cm',
  'cloudcover': '%'},
 'hourly': {'time': ['2018-10-25T00:00',
   '2018-10-25T01:00',
   '2018-10-25T02:00',
   '2018-10-25T03:00',
   '2018-10-25T04:00',
   '2018-10-25T05:00',
   '2018-10-25T06:00',
   '2018-10-25T07:00',
   '2018-10-25T08:00',
   '2018-10-25T09:00',
   '2018-10-25T10:00',
   '2018-10-25T11:00',
   '2018-10-25T12:00',
   '2018-10-25T13:00',
   '2018-10-25T14:00',
   '2018-10-25T15:00',
   '2018-10-25T16:00',
   '2018-10-25T17:00',
   '2018-10-25T18:00',
   '2018-10-25T19:00',
   '2018-10-25T20:00',
   '2018-10-25T21:00',
   '2018-10-25T22:00',
   '2018-10-25T23:00'],
  'precipitation': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
 

In [32]:
#test city scubber
city_scrubber("Dallas/Fort Something, TX")

'Dallas, TX'

In [33]:
city_scrubber("Austin, TX")

'Austin, TX'

In [34]:
city_scrubber("Austin")

'no state in your city string'

In [35]:
#test weather condition function
weather_condition("Denver, CO", "2019-11-23", latlong_df)

'sunny'

In [36]:
#test loop to fill df with weather condition
for row in range(3):
    city_date_unique['weather_categ'][row] = weather_condition(city_date_unique['city'][row], city_date_unique['fl_date'][row], latlong_df)

In [37]:
city_date_unique['weather_categ']

0        sunny
1         rain
2         rain
3        empty
4        empty
         ...  
15625    empty
15626    empty
15627    empty
15628    empty
15629    empty
Name: weather_categ, Length: 15630, dtype: object

### Run the function on the unique city/date combo table

In [ ]:
#BRANDON RAN THE FUNCTION AND SAVED THE RESULTS INTO final_unique_city
#should inlcude those cells here
#final_city_unique.csv

In [58]:
#load unique table with the weather data
city_unique = pd.read_csv('data/final_city_unique.csv', index_col=0)

In [59]:
city_unique.head(2)

,fl_date,city,count,weather_categ
0,2018-10-25,"New York, NY",7,sunny
1,2019-07-10,"Chicago, IL",7,rain


In [60]:
#check for null
city_unique['weather_categ'].isna().sum()

105

In [61]:
#fill Nans
city_unique['weather_categ'] = city_unique['weather_categ'].fillna('empty')

In [62]:
city_unique.groupby(by = 'weather_categ').count()

,fl_date,city,count
weather_categ,,,
cloudy,3168,3168,3168
empty,105,105,105
no weather data,77,77,77
rain,4532,4532,4532
snow,239,239,239
snow & rain,145,145,145
sunny,7364,7364,7364


Let's see which ones returned 'empty'. We know these are likely cities that were missing lat/long values

In [63]:
missing_latlong = city_unique[city_unique['weather_categ'] == 'empty'].groupby(by = 'city').count().reset_index()
missing_latlong 

,city,fl_date,count,weather_categ
0,"Ashland, WV",3,3,3
1,"Barrow, AK",3,3,3
2,"Charlotte Amalie, VI",9,9,9
3,"Christiansted, VI",4,4,4
4,"Deadhorse, AK",4,4,4
5,"Guam, TT",3,3,3
6,"Hilton Head, SC",2,2,2
7,"Islip, NY",15,15,15
8,"Kona, HI",44,44,44
9,"Lanai, HI",4,4,4


Looks like we were missing the lat/longs for these 14 cities. Let's find the lat/longs and create a new lat/long df

In [44]:
latlong_dict = {'city': list(missing_latlong ['city']), 
                'lat': [37.4084, 71.2906, 18.3419, 17.7466, 70.2002, 13.4443, 32.2163, 40.7298, 19.6419, 20.8166, 41.3805, 41.2835, 41.512177, 15.1850], 
                'lng': [-81.3526, -156.7886, -64.9307, -64.7032, -148.4597, 144.7937, -80.7526, -73.2104, -155.9962, -156.9273, -70.6455, -70.0995, -74.018326, 145.7467]}
extra_latlong = pd.DataFrame(latlong_dict)
extra_latlong

,city,lat,lng
0,"Ashland, WV",37.408400,-81.352600
1,"Barrow, AK",71.290600,-156.788600
2,"Charlotte Amalie, VI",18.341900,-64.930700
3,"Christiansted, VI",17.746600,-64.703200
4,"Deadhorse, AK",70.200200,-148.459700
5,"Guam, TT",13.444300,144.793700
6,"Hilton Head, SC",32.216300,-80.752600
7,"Islip, NY",40.729800,-73.210400
8,"Kona, HI",19.641900,-155.996200
9,"Lanai, HI",20.816600,-156.927300


In [45]:
#rename city to city_state
extra_latlong.rename({'city': 'city_state'}, axis = 1, inplace= True)

In [46]:
#filter the flights df so we only have to run through these cities
extra_flights_unique = city_unique[city_unique['weather_categ'] == 'empty']

In [47]:
extra_flights_unique.head()

,fl_date,city,count,weather_categ
1250,2019-10-06,"Kona, HI",2,empty
3471,2019-05-29,"Christiansted, VI",1,empty
3578,2019-06-11,"Kona, HI",1,empty
3620,2019-06-12,"Lanai, HI",1,empty
3697,2019-02-20,"Christiansted, VI",1,empty


In [48]:
#reset index
extra_flights_unique.reset_index(inplace = True)

We need to scrub the latlong city_states
This is because our weather_condition function will scrub the flights city name, but our latlong city is not scrubed yet so they won't match.

In [49]:
for row in range(len(extra_latlong)):
    extra_latlong['city_state'][row] = city_scrubber(extra_latlong['city_state'][row])

In [50]:
from tqdm import tqdm   

In [51]:
#run the function
for row in tqdm(range(len(extra_flights_unique))):
    extra_flights_unique['weather_categ'][row] = weather_condition(extra_flights_unique['city'][row], extra_flights_unique['fl_date'][row], extra_latlong)

100%|████████████████████████████████████████████████████████████████████████████████| 105/105 [00:38<00:00,  2.73it/s]


In [65]:
extra_flights_unique.drop('index', axis = 1, inplace = True)

In [67]:
extra_flights_unique

,fl_date,city,count,weather_categ
0,2019-10-06,"Kona, HI",2,sunny
1,2019-05-29,"Christiansted, VI",1,no weather data
2,2019-06-11,"Kona, HI",1,rain
3,2019-06-12,"Lanai, HI",1,no weather data
4,2019-02-20,"Christiansted, VI",1,no weather data
...,...,...,...,...
100,2018-10-19,"Charlotte Amalie, VI",1,no weather data
101,2018-11-04,"Islip, NY",1,sunny
102,2018-09-27,"Kona, HI",1,rain
103,2018-09-23,"Kona, HI",1,rain


In [66]:
extra_flights_unique['weather_categ'].isna().sum()

0

Great there are no more empty rows!

In [54]:
extra_flights_unique.groupby(by = 'weather_categ').count()

,index,fl_date,city,count
weather_categ,,,,
cloudy,6,6,6,6
no weather data,32,32,32,32
rain,49,49,49,49
snow & rain,1,1,1,1
sunny,17,17,17,17


In [69]:
# add the extra flights to the final_city_unique
city_unique  = city_unique.append(extra_flights_unique, ignore_index = True)

In [70]:
city_unique

,fl_date,city,count,weather_categ
0,2018-10-25,"New York, NY",7,sunny
1,2019-07-10,"Chicago, IL",7,rain
2,2019-10-02,"Chicago, IL",7,rain
3,2019-07-01,"Chicago, IL",7,cloudy
4,2019-11-23,"Denver, CO",7,sunny
...,...,...,...,...
15730,2018-10-19,"Charlotte Amalie, VI",1,no weather data
15731,2018-11-04,"Islip, NY",1,sunny
15732,2018-09-27,"Kona, HI",1,rain
15733,2018-09-23,"Kona, HI",1,rain


In [71]:
city_unique.groupby(by = 'weather_categ').count()

,fl_date,city,count
weather_categ,,,
cloudy,3174,3174,3174
empty,105,105,105
no weather data,109,109,109
rain,4581,4581,4581
snow,239,239,239
snow & rain,146,146,146
sunny,7381,7381,7381


In [72]:
#drop those empty rows, they are now duplicates
city_unique = city_unique[city_unique['weather_categ'] != 'empty']

In [79]:
city_unique.groupby(by = 'weather_categ').count()

,fl_date,city,count
weather_categ,,,
cloudy,3174,3174,3174
no weather data,109,109,109
rain,4581,4581,4581
snow,239,239,239
snow & rain,146,146,146
sunny,7381,7381,7381


In [74]:
city_unique.to_csv('data/final_city_unique.csv', header=True, index=False)

## Now we can merge the weather data onto the flights df

### Match the weather to the flights

In [54]:
#load flights data
flights = pd.read_csv('data/flights_10000.csv', index_col=0)

In [80]:
flights.head(2)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,orig_weather_categ,dest_weather_categ
0,2018-07-10,WN,WN,WN,2212,WN,N958WN,2212,14683,SAT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cloudy,cloudy
1,2019-10-10,UA,UA,UA,206,UA,N68811,206,13930,ORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rain,rain


In [81]:
#we don't want to add the count
city_unique.drop('count', axis = 1, inplace = True)

In [82]:
flights.drop(['orig_weather_categ','dest_weather_categ'], axis = 1, inplace= True)

In [83]:
#merge for origin city
flights = pd.merge(flights, city_unique, how = 'left', left_on = ['fl_date', 'origin_city_name'], right_on = ['fl_date', 'city'])

In [84]:
flights.head(2)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,city,weather_categ
0,2018-07-10,WN,WN,WN,2212,WN,N958WN,2212,14683,SAT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"San Antonio, TX",cloudy
1,2019-10-10,UA,UA,UA,206,UA,N68811,206,13930,ORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Chicago, IL",rain


In [85]:
#rename col weather_origin
flights.rename({'weather_categ': 'orig_weather_categ'}, axis = 1, inplace= True)
#drop the city col which also merged
flights.drop('city', axis = 1, inplace = True)

In [86]:
flights.head(2)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,orig_weather_categ
0,2018-07-10,WN,WN,WN,2212,WN,N958WN,2212,14683,SAT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cloudy
1,2019-10-10,UA,UA,UA,206,UA,N68811,206,13930,ORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rain


In [87]:
#add the dest weather
flights = pd.merge(flights, city_unique, how = 'left', left_on = ['fl_date', 'dest_city_name'], right_on = ['fl_date', 'city'])

In [88]:
flights.head(2)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,orig_weather_categ,city,weather_categ
0,2018-07-10,WN,WN,WN,2212,WN,N958WN,2212,14683,SAT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cloudy,"Nashville, TN",cloudy
1,2019-10-10,UA,UA,UA,206,UA,N68811,206,13930,ORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rain,"Orlando, FL",rain


In [89]:
#rename col weather dest
flights.rename({'weather_categ': 'dest_weather_categ'}, axis = 1, inplace= True)
#drop the city col which also merged
flights.drop('city', axis = 1, inplace = True)

In [90]:
flights.head(2)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,orig_weather_categ,dest_weather_categ
0,2018-07-10,WN,WN,WN,2212,WN,N958WN,2212,14683,SAT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cloudy,cloudy
1,2019-10-10,UA,UA,UA,206,UA,N68811,206,13930,ORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rain,rain


In [91]:
#save over the flights_10000 csv
flights.to_csv('data/flights_10000.csv', header=True, index=False)